# Training and A/B test for Conversion agents - <font color='blue'> Presence</font> of sale

In [1]:
import gym, recogym
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import pandas as pd
import pickle as pkl
pd.options.mode.chained_assignment = None 
# from inspect import getsource
from recogym.evaluate_agent_sale import verify_agents_sale, display_metrics
from tqdm import tqdm

# env_1_sale_args is a dictionary of default parameters (i.e. number of products)
from recogym import env_1_sale_args, Configuration
from recogym.agents.sale_agent import train_agents, train_timeagents
from recogym.envs.utils_sale import format_avg_result, avg_result


# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 0
env_1_sale_args['num_products'] = 10
env_1_sale_args['number_of_flips'] = 10 

num_products = env_1_sale_args['num_products']
print('Number of products =',num_products)
print('Number of flips =',env_1_sale_args['number_of_flips'])
nb_flips = env_1_sale_args['number_of_flips']

# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 42
env_1_sale_args['mu_sale'] = False 

# env_1_sale_args['kappa'] = 0.5
print('Value of kappa =',env_1_sale_args['kappa'])

# Initialize the gym 
env = gym.make('reco-gym-sale-v1')
env.init_gym(env_1_sale_args)



Number of products = 10
Number of flips = 10
Value of kappa = 0.2


In [2]:
# Repository to save pickles
data_repo = 'data_conversion/'

## Train baseline agents

#### Settings

In [3]:
## Choose number of users for training and AB test
# Number of users for the training
# env_1_sale_args['num_users'] = 6 ##tochange !!
env_1_sale_args['num_users'] = 5000
num_users = env_1_sale_args['num_users']

# Number of users for the A/B test
# env_1_sale_args['num_users_AB'] = 7 ##tochange !!
env_1_sale_args['num_users_AB'] = 5000
num_users_AB = env_1_sale_args['num_users_AB']

# Choose user features
from recogym.agents.sale_agent import CountViewsClicksFeatureProvider, CountViewsFeatureProvider, ShareViewsClicksFeatureProvider, ShareViewsFeatureProvider
vc_feature = CountViewsClicksFeatureProvider(env.config)
v_feature = CountViewsFeatureProvider(env.config)
vc_share_feature = ShareViewsClicksFeatureProvider(env.config)
v_share_feature = ShareViewsFeatureProvider(env.config)
features = {'vc':vc_feature,
           'v':v_feature,
           'vc_share':vc_share_feature,
           'v_share':v_share_feature}
feature_name = 'v_share'
feature = features[feature_name]

#### <font color='red'> Number of A/B tests</font>

In [4]:
# # Choose number of A/B tests
# num_AB_tests = 2 ##tochange !
num_AB_tests = 25 ##tochange !

#### Logs

In [5]:
agents={}
logs={}

############## Random agent
name_agent = 'rand'+str(nb_flips)

try:
    logs[name_agent] = pkl.load(open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','rb'))
    print('--- Logs loaded---')
except: 
    print("--- Generate logs ---")
    from recogym.agents import RandomAgent, random_args
    random_agent = RandomAgent(Configuration(random_args))
    agents[name_agent] = random_agent
    logs[name_agent] = deepcopy(env).generate_logs(num_users)
    print(data_repo + 'data' + str(num_users) + name_agent + '.pkl')
    pkl.dump(logs[name_agent], open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','wb'))

--- Logs loaded---


#### Training for the <font color='blue'> Presence</font> of sale

##### No weights

In [6]:
name_extension = 'pres'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('agents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_agents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,additional_models_only=False, repo = data_repo)

--- Trained agents loaded --- 


In [7]:
try:
    for i in range(num_AB_tests):
        name_extension = 'pres'+'_nb'+str(i)
        name = 'likelihood_saleclickprod_discount_allpres'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        print(str(data_repo) +'res_'+agent_id+name+'.pkl')
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'pres'+'_nb'+str(i)
        res=verify_agents_sale(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

data_conversion/res_rand106_7_v_sharepres_nb0likelihood_saleclickprod_discount_allpres.pkl
data_conversion/res_rand106_7_v_sharepres_nb1likelihood_saleclickprod_discount_allpres.pkl
--- A/B test results loaded ---


In [8]:
res_avg = avg_result(res_dict)
df, df_latex = format_avg_result(res_avg) #get dataframe & corresponding latex table

,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Click,0.929 (0.307) %,1 (1),13.04 (4.22) %,7 (0),30.98 (4.23) %
1,PDS,0.758 (0.032) %,1 (0),16.03 (8.45) %,5 (1),25.00 (4.23) %
2,PCS,0.589 (0.003) %,1 (0),16.03 (8.45) %,6 (1),22.01 (8.46) %
3,DPCS,0.517 (0.164) %,0 (0),13.04 (4.22) %,4 (3),22.01 (8.46) %


In [9]:
# display(df)
print(df_latex)

\begin{tabular}{llllll}
\toprule
 Agent &              CTR & Att Sales &          Att CR &  Sales &              CR \\
\midrule
 Click &  0.929 (0.307) \% &     1 (1) &  13.04 (4.22) \% &  7 (0) &  30.98 (4.23) \% \\
   PDS &  0.758 (0.032) \% &     1 (0) &  16.03 (8.45) \% &  5 (1) &  25.00 (4.23) \% \\
   PCS &  0.589 (0.003) \% &     1 (0) &  16.03 (8.45) \% &  6 (1) &  22.01 (8.46) \% \\
  DPCS &  0.517 (0.164) \% &     0 (0) &  13.04 (4.22) \% &  4 (3) &  22.01 (8.46) \% \\
\bottomrule
\end{tabular}



##### Weights

In [10]:
name_extension = 'presweights'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('agents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_agents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,weights = True, additional_models_only=False, repo = data_repo)

--- Trained agents loaded --- 


In [11]:
try:
    for i in range(num_AB_tests):
        name_extension = 'presweights'+'_nb'+str(i)
        name = 'likelihood_saleclickprod_discount_allpresweights'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'presweights'+'_nb'+str(i)
        res=verify_agents_sale(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

--- A/B test results loaded ---


In [12]:
res_avg = avg_result(res_dict)
df, df_latex = format_avg_result(res_avg) #get dataframe & corresponding latex table

,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Click,0.798 (0.633) %,0 (1),13.08 (12.63) %,2 (2),19.02 (12.68) %
1,PDS,0.701 (0.240) %,0 (0),13.04 (4.22) %,5 (2),30.98 (12.68) %
2,PCS,0.443 (0.036) %,1 (0),13.04 (4.22) %,7 (7),27.98 (16.91) %
3,DPCS,0.822 (0.339) %,2 (0),22.01 (0.00) %,11 (0),27.99 (0.00) %


In [13]:
# display(df)
print(df_latex)

\begin{tabular}{llllll}
\toprule
 Agent &              CTR & Att Sales &           Att CR &   Sales &               CR \\
\midrule
 Click &  0.798 (0.633) \% &     0 (1) &  13.08 (12.63) \% &   2 (2) &  19.02 (12.68) \% \\
   PDS &  0.701 (0.240) \% &     0 (0) &   13.04 (4.22) \% &   5 (2) &  30.98 (12.68) \% \\
   PCS &  0.443 (0.036) \% &     1 (0) &   13.04 (4.22) \% &   7 (7) &  27.98 (16.91) \% \\
  DPCS &  0.822 (0.339) \% &     2 (0) &   22.01 (0.00) \% &  11 (0) &   27.99 (0.00) \% \\
\bottomrule
\end{tabular}

